In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt 
import math
import random
import tensorflow as tf
from imutils import paths
from tensorflow.keras import layers
from PIL import Image

In [ ]:
def plot_images(img1, img2,img3, 
                img4, img5,img6,  
                img7, img8,img9,
                title1="45 degree", title2="Original", title3="315 degree",
                title4="blur 45 degree", title5="blur Original", title6="blur 315 degree",
                title7="contrast 45 degree", title8="contrast Original", title9="contrast 315 degree"):
    fig = plt.figure(figsize=[15,15])

    ax1 = fig.add_subplot(331)
    ax1.imshow(img1, cmap="Accent")
    ax1.set(xticks=[], yticks=[], title=title1)

    ax2 = fig.add_subplot(332)
    ax2.imshow(img2, cmap="Accent")
    ax2.set(xticks=[], yticks=[], title=title2)

    ax3 = fig.add_subplot(333)
    ax3.imshow(img3, cmap="Accent")
    ax3.set(xticks=[], yticks=[], title=title3)

    #-----

    ax4 = fig.add_subplot(334)
    ax4.imshow(img4, cmap="Accent")
    ax4.set(xticks=[], yticks=[], title=title4)

    ax5 = fig.add_subplot(335)
    ax5.imshow(img5, cmap="Accent")
    ax5.set(xticks=[], yticks=[], title=title5)

    ax6 = fig.add_subplot(336)
    ax6.imshow(img6, cmap="Accent")
    ax6.set(xticks=[], yticks=[], title=title6)

    #-----

    ax7 = fig.add_subplot(337)
    ax7.imshow(img7, cmap="Accent")
    ax7.set(xticks=[], yticks=[], title=title7)

    ax8 = fig.add_subplot(338)
    ax8.imshow(img8, cmap="Accent")
    ax8.set(xticks=[], yticks=[], title=title8)

    ax9 = fig.add_subplot(339)
    ax9.imshow(img9, cmap="Accent")
    ax9.set(xticks=[], yticks=[], title=title9)

def rotate(img,w,h,angle):
  center = (w / 2, h / 2)

  scale = 1.0

  M = cv2.getRotationMatrix2D(center, angle, scale)
  rotated = cv2.warpAffine(img, M, (h, w))
  return rotated

def gaussian(img,i):
  i = int(i)
  blur = cv2.blur(img,(i,i))
  return blur

def brightness(img, i):
  i = int(i)
  kernel = np.ones((i,i),np.float32)/25
  dst = cv2.filter2D(img,-1,kernel)
  return dst

def noise_image(img):
  # Generate Gaussian noise
  gauss = np.random.normal(0,1,img.size)
  gauss = gauss.reshape(img.shape[0],img.shape[1],img.shape[2]).astype('uint8')
  # Add the Gaussian noise to the image
  img_gauss = cv2.add(img,gauss)
  return img_gauss

In [ ]:
# read image paths and labels
paths = list(paths.list_images('/content/drive/MyDrive/Object Detection/no roboflow/data_preparation/original_image'))

In [ ]:
w = 1000
h = 1000
dim = (w, h)

def plot_2_images(img1,
                title1=""):
    fig = plt.figure(figsize=[15,15])

    ax1 = fig.add_subplot(111)
    ax1.imshow(img1, cmap="Accent")
    ax1.set(xticks=[], yticks=[], title=title1)

def plot_3_images(img1, img2, img3,
                title1="s&p 1", title2="s&p 2", title3 = "s&p 3"):
    fig = plt.figure(figsize=[15,15])

    ax1 = fig.add_subplot(131)
    ax1.imshow(img1, cmap="Accent")
    ax1.set(xticks=[], yticks=[], title=title1)

    ax2 = fig.add_subplot(132)
    ax2.imshow(img2, cmap="Accent")
    ax2.set(xticks=[], yticks=[], title=title2)

    ax3 = fig.add_subplot(133)
    ax3.imshow(img3, cmap="Accent")
    ax3.set(xticks=[], yticks=[], title=title3)

filename = '/content/image (100).jpg'
img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
#plot_2_images(img,resized)

rotated45 = rotate(resized,w,h,45)
rotated315 = rotate(resized,w,h,315)

#resized = noise_image(resized)
rotated45 = noise_image(rotated45)
rotated315 = noise_image(rotated315)

plot_2_images(resized)

In [ ]:
w = 1000
h = 1000
dim = (w, h)

filename = '/content/drive/MyDrive/Object Detection/no roboflow/data_preparation/rotated_image/plate_number_augment_'
count = 1

for p in paths:
  img = cv2.imread(p, cv2.IMREAD_UNCHANGED)

  # augment image

  #--resize image
  resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

  #--rotate image
  rotated45 = rotate(resized,w,h,45)
  rotated315 = rotate(resized,w,h,315)

  #--blur image
  i = 17
  i = random.choice(i)
  blur = gaussian(resized,i)
  blur45 = gaussian(rotated45,i)
  blur315 = gaussian(rotated315,i)  

  #--contrast
  i = '27'
  i = random.choice(i)
  contrast = brightness(resized,i)
  contrast45 = brightness(rotated45,i)
  contrast315 = brightness(rotated315,i)

  #--salt and pepper noise

  resized = noise_image(resized)
  rotated45 = noise_image(rotated45)
  rotated315 = noise_image(rotated315)

  # plot images
  plot_images(rotated45, resized, rotated315,
              blur45, blur, blur315, 
              contrast45, contrast, contrast315)

  # save image
  count_str = str(count)  

  #--rotate
  cv2.imwrite(filename+count_str+'_ori'+'.jpg', resized)
  cv2.imwrite(filename+count_str+'_rotated45'+'.jpg', rotated45) 
  cv2.imwrite(filename+count_str+'_rotated315'+'.jpg', rotated315) 
  
  #--blur
  cv2.imwrite(filename+count_str+'_blur'+'.jpg', blur)
  cv2.imwrite(filename+count_str+'_blur45'+'.jpg', blur45) 
  cv2.imwrite(filename+count_str+'_blur315'+'.jpg', blur315) 

  #--contrast
  cv2.imwrite(filename+count_str+'_contrast'+'.jpg', contrast)
  cv2.imwrite(filename+count_str+'_contrast45'+'.jpg', contrast45) 
  cv2.imwrite(filename+count_str+'_contrast315'+'.jpg', contrast315) 
  
  count = count + 1
  #break
